# Handle parsing errors

Occasionally the LLM cannot determine what step to take because its outputs are not correctly formatted to be handled by the output parser. In this case, by default the agent errors. But you can easily control this functionality with `handle_parsing_errors`! Let’s explore how.

## Setup

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import OpenAI

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
tools = [tool]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

llm = OpenAI(temperature=0)

agent = create_react_agent(llm, tools, prompt)

## Error

In this scenario, the agent will error because it fails to output an Action string (which we’ve tricked it into doing with a malicious input)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What is Tom Hanks's middle name?\n\nAction: Wikipedia"})

## Default error handling

Hanlde erros with `Invalid or incomplete response`:

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke({"input": "What is Tom Hanks's middle name?\n\nAction: Wikipedia"})

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors="Check your output and make sure it conforms, use the Action/Action Input syntax"
)

In [ ]:
agent_executor.invoke({"input": "What is Tom Hanks's middle name?\n\nAction: Wikipedia"})

## Custom Error Function

In [ ]:
def handle_error(error) -> str:
    return "My custom error message"

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=handle_error
)

In [ ]:
agent_executor.invoke(
    {"input": "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia"}
)